- Assesment on Spam Detection Project
   - Collect data from kaggle and download required data.
   - Use same code on downloaded dataset.
   - go for tts model by 80%(train) and 20%(test)
   - Need to research on how to built a GUI for above ML model using tkinter

In [3]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import  train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

In [4]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
stemmer = PorterStemmer()
stop_words = set(stopwords.words("english"))

In [6]:
df = pd.read_csv("spam_ham_dataset.csv",encoding='latin-1')[['label','text']]

In [7]:
df.head()

,label,text
0,ham,Subject: enron methanol ; meter # : 988291\nth...
1,ham,"Subject: hpl nom for january 9 , 2001\n( see a..."
2,ham,"Subject: neon retreat\nho ho ho , we ' re arou..."
3,spam,"Subject: photoshop , windows , office . cheap ..."
4,ham,Subject: re : indian springs\nthis deal is to ...


In [8]:
df.describe()

,label,text
count,5171,5171
unique,2,4993
top,ham,Subject: calpine daily gas nomination\n>\nrick...
freq,3672,20


In [9]:
df.isna().sum()

label    0
text     0
dtype: int64

In [10]:
# 0 = ham
# 1 = spam
df['label'] = df['label'].map({"ham":0,"spam":1})

In [11]:
df.head()

,label,text
0,0,Subject: enron methanol ; meter # : 988291\nth...
1,0,"Subject: hpl nom for january 9 , 2001\n( see a..."
2,0,"Subject: neon retreat\nho ho ho , we ' re arou..."
3,1,"Subject: photoshop , windows , office . cheap ..."
4,0,Subject: re : indian springs\nthis deal is to ...


In [12]:
df.columns = ['lable',"message"]

In [13]:
def preprocess_text(text):
    text = re.sub(r"\W", " ", text) # remove special symbol
    text = text.lower() # all text to lower case
    words = text.split() # collect words in list
    words = [stemmer.stem(word) for word in words if word not in stop_words] # remove stopewords and stem words
    return " ".join(words)

In [14]:
df["cleaned_message"] = df["message"].apply(preprocess_text)

In [15]:
df.head()

,lable,message,cleaned_message
0,0,Subject: enron methanol ; meter # : 988291\nth...,subject enron methanol meter 988291 follow not...
1,0,"Subject: hpl nom for january 9 , 2001\n( see a...",subject hpl nom januari 9 2001 see attach file...
2,0,"Subject: neon retreat\nho ho ho , we ' re arou...",subject neon retreat ho ho ho around wonder ti...
3,1,"Subject: photoshop , windows , office . cheap ...",subject photoshop window offic cheap main tren...
4,0,Subject: re : indian springs\nthis deal is to ...,subject indian spring deal book teco pvr reven...


In [16]:
vectorizer = TfidfVectorizer(max_features=3000)
X = vectorizer.fit_transform(df["cleaned_message"]) # input data
y = df["lable"] # output data

In [17]:
X_train,X_test,y_train,y_test = train_test_split(X, y, train_size = 0.8, random_state = 42)

In [18]:
model = LogisticRegression()

In [19]:
model.fit(X_train,y_train) # trained Model we can predict test data

LogisticRegression()

In [20]:
y_pred = model.predict(X_test)

In [21]:
print(f"Accuracy : {accuracy_score(y_test, y_pred) * 100:.2f}%")

Accuracy : 98.26%


In [22]:
print(f"Classification report :\n\n{classification_report(y_test,y_pred)}")

Classification report :

              precision    recall  f1-score   support

           0       0.99      0.99      0.99       742
           1       0.97      0.97      0.97       293

    accuracy                           0.98      1035
   macro avg       0.98      0.98      0.98      1035
weighted avg       0.98      0.98      0.98      1035



In [23]:
def predict_email(email_text):
    processed_data = preprocess_text(email_text)
    vectorized_text = vectorizer.transform([processed_data])
    prediction = model.predict(vectorized_text)
    return "Spam" if prediction[0]==1 else "Ham - Not Spam"
    

In [24]:
email = "Will ﾌ_ b going to esplanade fr home?"

In [25]:
print(f"Email : {email}\n\nPrediction : {predict_email(email)}")

Email : Will ﾌ_ b going to esplanade fr home?

Prediction : Spam
